<a href="https://colab.research.google.com/github/Radperia/SIGNATE_student_2020/blob/master/sklearn_stacking_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

いつもの時間確認

In [2]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 / 60 / 60 "h)"}'

0.000965741days (0.0231778h)


色々importとcatboost, light gbmのインストール

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import gc

In [6]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 65.9MB 60kB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [7]:
!git clone https://github.com/microsoft/LightGBM
!cd LightGBM && mkdir build && cd build && cmake .. && make -j"$(nproc)"
!cd LightGBM/python-package && python setup.py install --precompile

Cloning into 'LightGBM'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 18879 (delta 6), reused 8 (delta 2), pack-reused 18855
Receiving objects: 100% (18879/18879), 12.71 MiB | 25.21 MiB/s, done.
Resolving deltas: 100% (13794/13794), done.
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found Op

In [31]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [9]:
NFOLD = 5
SEED = 42

データの読み込み

In [41]:
train_df = pd.read_csv('./drive/My Drive/SIGNATE/train.csv').fillna(' ')
test_df  = pd.read_csv('./drive/My Drive/SIGNATE/test.csv').fillna(' ')
sample = pd.read_csv('./drive/My Drive/SIGNATE/submit_sample.csv', header=None)

train_text = train_df['description']
test_text = test_df['description']
all_text = pd.concat([train_text, test_text])

sentences = train_df['description'].tolist() + test_df['description'].tolist()
sentences[::500]  # 500個飛ばしに確認

['Executes and writes portions of testing plans, protocols, and documentation for assigned portion of application; identifies and debugs issues with code and suggests changes or improvements.',
 'Knowledge and experience of complex software design for distributed systems in embedded networking/telecommunications projects.',
 'Analyze client business processes of storing and managing data.',
 'Defining tasks, timeline and required resources to deliver on the mechanical design portion of a project brief, with strong accountability for successful completion of agreed upon deliverables.',
 'Familiar with sensors, transducers, physiologic modeling, and transfer functions',
 'Manage, maintain, refresh and upgrade environmental components (including patching, etc.)',
 'Conduct research on cutting-edge techniques and tools in machine learning/deep learning/artificial intelligence',
 'Assist in planning and development of visualizations and analytic aggregations to best portray discovered insig

In [11]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000)  # 出現頻度上位{num_words}だけを用いる
tokenizer.fit_on_texts(sentences)
print(list(tokenizer.word_index)[:10] ) # 学習された辞書（出現頻度順）

['and', 'to', 'the', 'of', 'with', 'data', 'in', 'for', 'a', 'business']


In [13]:
# 学習・検証に分けて，ベクトルを生成。加えて，正解ラベルも作成
train_X, test_X = np.split(tokenizer.texts_to_matrix(sentences, mode='binary'),
                           [len(train_df)], axis=0)

train_y = train_df['jobflag'].values - 1  # maps {1, 2, 3 ,4} -> {0, 1, 2, 3}
train_X.shape, train_y.shape, test_X.shape

((2931, 1000), (2931,), (1743, 1000))

参考にした

https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm

In [14]:
kf = KFold(n_splits = NFOLD, shuffle=True, random_state=SEED)

In [22]:
ntrain = train_X.shape[0]
ntest = test_X.shape[0]

In [16]:
class SklearnWrapper(object):
  def __init__(self, clf, seed=SEED, params=None):
    params['random_state'] = seed
    self.clf = clf(**params)

  def train(self, train_X, train_y):
    self.clf.fit(train_X, train_y)

  def predict(self, x):
    return self.clf.predict_proba(x)[:, 1]  

In [19]:
class CatboostWrapper(object):
  def __init__(self, clf, seed=SEED, params=None):
    params['random_seed'] = seed
    self.clf = clf(**params)

  def train(self, train_X, train_y):
    self.clf.fit(train_X, train_y)

  def predict(self, x):
    return self.clf.predict_proba(x)[:, 1]

In [57]:
class LightGBMWrapper(object):
  def __init__(self, clf, seed=SEED, params=None):
    params['feature_fraction_seed'] = seed
    params['bagging_seed'] = seed
    self.clf = clf(**params)

  def train(self, train_X, train_y):
    self.clf.fit(train_X, train_y)
  
  def predict(self, x):
    return self.clf.predict_proba(x)[:, 1]

In [21]:
class XgbWrapper(object):
  def __init__(self, seed=SEED, params=None):
    self.param = params
    self.param['seed'] = seed
    self.nrounds = params.pop('nrounds', 250)

  def train(self, train_X, train_y):
    dtrain = xgb.DMatrix(train_X, label=train_y)
    self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

  def predict(self, x):
    return self.gbdt.predict(xgb.DMatrix(x))

In [43]:
def get_oof(clf):
  oof_train = np.zeros((ntrain,))
  oof_test = np.zeros((ntest,))
  oof_test_skf = np.empty((NFOLD, ntest))

  for i, (train_idx, test_idx) in enumerate(kf.split(train_X)):
    x_tr = train_X[train_idx]
    y_tr = train_y[train_idx]
    x_te = train_X[test_idx]

    clf.train(x_tr, y_tr)

    oof_train[test_idx] = clf.predict(x_te)
    oof_test_skf[i, :] = clf.predict(test_X)

  oof_test[:] = oof_test_skf.mean(axis=0)
  return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [25]:
et_params = {
    'n_jobs': -1,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf':2
}

In [26]:
rf_params = {
    'n_jobs': -1,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2
}

In [28]:
xgb_params = {
    'seed': SEED,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

In [33]:
catboost_params = {
    'iterations': 200,
    'learning_rate': 0.5,
    'depth': 3,
    'l2_leaf_reg': 40,
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'allow_writing_files': False
}

In [30]:
lightgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.1,
    'num_leaves': 123,
    'colsample_bytree': 0.8,
    'subsample': 0.9,
    'max_depth': 15,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'min_split_gain': 0.01,
    'min_child_weight': 2
}

In [55]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf=CatBoostClassifier, seed=SEED, params=catboost_params)
lg = LightGBMWrapper(clf=LGBMClassifier, seed=SEED, params=lightgbm_params)

In [45]:
et_oof_train, et_oof_test = get_oof(et)

In [46]:
rf_oof_train, rf_oof_test = get_oof(rf)

CatBoostとXGBでバグってる(out of foldが合わない)

In [47]:
cb_oof_train, cb_oof_test = get_oof(cb)

CatBoostError: ignored

In [48]:
xg_oof_train, xg_oof_test = get_oof(xg)

XGBoostError: ignored

In [56]:
lg_oof_train, lg_oof_test = get_oof(lg)

[LightGBM] [Warning] Unknown parameter: baggin_seed
[LightGBM] [Warning] Unknown parameter: baggin_seed
[LightGBM] [Warning] Unknown parameter: baggin_seed
[LightGBM] [Warning] Unknown parameter: baggin_seed
[LightGBM] [Warning] Unknown parameter: baggin_seed


In [62]:
#print("XG-CV: {}".format(sqrt(mean_squared_error(train_y, xg_oof_train))))
print("ET-CV: {}".format(sqrt(mean_squared_error(train_y, et_oof_train))))
print("RF-CV: {}".format(sqrt(mean_squared_error(train_y, rf_oof_train))))
#print("CB-CV: {}".format(sqrt(mean_squared_error(train_y, cb_oof_train))))
print("LG-CV: {}".format(sqrt(mean_squared_error(train_y, lg_oof_train))))

ET-CV: 1.8786292311560924
RF-CV: 1.8736198759116647
LG-CV: 1.883637116226478
